# 75.70 Sistemas de Programación no Convencional de Robots

# Trabajo Práctico Final

# 1er Cuatrimestre 2019

## Grupo 2

### Martin Stancanelli 95188
### Juan Manuel Lambre xxxxx

# Instalación de dependencias

In [1]:
!pip install tpot

     |████████████████████████████████| 81kB 4.1MB/s 
     |████████████████████████████████| 153kB 12.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit
